### Apply Keyword Extraction Using Staistical Method

In [1]:
import os
import pandas as pd
import numpy as np
import string

import pke

import spacy
from spacy import displacy
nlp = spacy.load('en')

from nltk import download
download('stopwords')
from nltk.corpus import stopwords

output_dir = '../Output/'
os.getcwd()

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'/mnt/Model'

#### Built stoplist

In [2]:
stoplist = list(string.punctuation)
stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
stoplist += stopwords.words('english')

#### Load Data

In [3]:
submissions = pd.read_excel(os.path.join(output_dir,'Submissions_test.xls'))

In [4]:
def extract_key_words(text, method, n_word = 10):
    '''
    input: text- string, method: string key for extractor_list
    output: list of top key words extracted
    '''
    extractor_dict = {
        ## statiscal methods:
        'tfidf': pke.unsupervised.TfIdf(),
        'kp miner': pke.unsupervised.KPMiner(),
        'YAKE': pke.unsupervised.YAKE()
    }
    
    df= pke.load_document_frequency_file(input_file='../Output/output.tsv.gz')
    extractor = extractor_dict[method]
    extractor.load_document(input= text, language='en', normalization='stemming')
    # keyphrase candidate selection
    if (method in('tfidf','kp miner')):
        extractor.candidate_selection(stoplist = stoplist)
        extractor.candidate_weighting(df= df)
    else:
        extractor.candidate_selection()
        extractor.candidate_weighting()

    keyphrases = extractor.get_n_best(n= n_word) # get key phrases
    
    return keyphrases

In [ ]:
import warnings
warnings.filterwarnings('ignore')

for key in ['tfidf', 'kp miner', 'YAKE']:
    print('Keyword extraction using key: ', key, r'...')
    submissions[key] = submissions.Description.apply(lambda x: extract_key_words(x, key, n_word= 5))

In [6]:
submissions.to_csv('../Output/Submissions_with_Keyword_Extraction_Statistical.csv')

#### Pick one for example

In [ ]:
i = 2
text = submissions.loc[i, 'Description']
print (submissions.loc[i,'Title_modified'])
#### Visuzliae Entity
doc = nlp(text)
displacy.render(doc, style='ent', jupyter= True)
#### Print out keywords Extracted
for key in ['tfidf', 'kp miner', 'YAKE']:
    print('Keyword extraction using: ', key, r'...')
    print(extract_key_words(text, key, n_word= 6), '\n')

#### *Need to run the followint to Create and Save Document Frequency File (in separate file)*

def save_document(name, text):
    with open(os.path.join(output_dir,'Document/', str(name)+'.txt'), "w") as text_file:
        text_file.write(text)

submissions[['Title_modified','Description']].apply(lambda x: save_document(x[0],x[1]), axis=1)

from pke import compute_document_frequency

compute_document_frequency?

compute_document_frequency(input_dir=os.path.join(output_dir,'Document'),
                           output_file= os.path.join(output_dir,'output.tsv.gz'),
                           extension='txt',           # input file extension
                           language='en',                # language of files
                           normalization="stemming",    # use porter stemmer
                           stoplist=stoplist)